In [20]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
from kerastuner import HyperModel
from tensorflow.keras import backend as K
import gradio as gr

In [21]:
# Define R2 since not in keras and utilized in model

def r_squared(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    SS_res = K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res / (SS_tot + K.epsilon()))

In [22]:
# Load in model 
model = keras.models.load_model('diamond_price_prediction_nn_model.keras', custom_objects={'r_squared': r_squared})

In [23]:
# Manually create mappings based on how LabelEncoders were fit
cut_map = {'Fair': 0, 'Good cut': 1, 'Very Good': 4, 'Premium': 3, 'Ideal': 2}
color_map = {'D': 0, 'E': 1, 'F': 2, 'G': 3, 'H': 4, 'I': 5, 'J': 6}
clarity_map = {'I1': 0, 'IF': 1, 'SI1': 2, 'SI2': 3, 'VS1': 4, 'VS2': 5, 'VVS1': 6, 'VVS2': 7}

In [24]:
# Define the min and max values from scaling done in model development
min_values = np.array([ 0.2, 0, 0, 0, 43, 43, 3.73, 3.68, 1.07])
range_values = np.array([4.81, 4, 6, 7, 36, 52, 7.01, 55.2, 30.73]) 

In [25]:
# Define prediction function
def predict(carat, cut, color, clarity, depth, table, x, y, z):
    # Safely get values from maps with a default if missing
    cut_encoded = cut_map.get(cut)
    color_encoded = color_map.get(color)
    clarity_encoded = clarity_map.get(clarity)

    inputs = np.array([[carat, cut_encoded, color_encoded, clarity_encoded, depth, table, x, y, z]])
    scaled_inputs = (inputs - min_values) / range_values
    prediction = model.predict(scaled_inputs)
    return prediction[0][0]

In [26]:
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label='carat', value=''),
        gr.Dropdown(label="cut", choices=list(cut_map.keys())),
        gr.Dropdown(label="color", choices=list(color_map.keys())),
        gr.Dropdown(label="clarity", choices=list(clarity_map.keys())),
        gr.Number(label="depth", value=''),
        gr.Number(label="table", value=''),
        gr.Number(label="x", value=''),
        gr.Number(label="y", value=''),
        gr.Number(label="z", value='')
    ],
    outputs=gr.Textbox(label="Predicted Price", lines=2),
    title="Diamond Price Predictor",
    description="Enter the attributes of the diamond to predict its price.",
    theme="huggingface"
)
iface.launch()

/Users/samuelvincentneal/anaconda3/envs/dev/lib/python3.10/site-packages/gradio/blocks.py:626: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-66206d27-70274cdc6c4c078a44e66f98;cb31188a-251e-404d-b4cf-9d1fae6a6c12)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
